- set min amount of space between objects, max amount of objects, iterations

- generate random position & rotation for objects -> AABB check in plane, check collision -> yes check SAT collision -> no check distance closest point -> > space between -> distance center of mass -> add from smallest to biggest


In [2]:
%pip install numpy plotly

Note: you may need to restart the kernel to use updated packages.


In [3]:
import subprocess, struct
import numpy as np
import plotly.graph_objects as go

In [4]:
def create_openscad_model(scad_code, filename):
    filename = f"{filename}.scad"
    # Write the code to an .scad file
    with open(filename, "w") as f:
        f.write(scad_code)

    print(f"{filename} created successfully.")


def convert_scad_to_stl(input_file, output_file):
    input_file = f"{input_file}.scad"
    output_file = f"{output_file}.stl"

    try:
        subprocess.check_call(["openscad", "-o", output_file, input_file])
        print(f"{input_file} successfully converted to {output_file}")
    except subprocess.CalledProcessError:
        print(f"Error converting {input_file} to {output_file}")

In [5]:
modelOneFileName = "modelOne"
modelTwoFileName = "modelTwo"

modelOneScadCode = "cube(1);"
modelTwoScadCode = "cube(1);"

create_openscad_model(modelOneScadCode, modelOneFileName)
create_openscad_model(modelTwoScadCode, modelTwoFileName)

convert_scad_to_stl(modelOneFileName, modelOneFileName)
convert_scad_to_stl(modelTwoFileName, modelTwoFileName)

modelOne.scad created successfully.
modelTwo.scad created successfully.
modelOne.scad successfully converted to modelOne.stl
modelTwo.scad successfully converted to modelTwo.stl


Geometries in cache: 1
Geometry cache size in bytes: 728
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.001
   Top level object is a 3D object:
   Facets:          6
Geometries in cache: 1
Geometry cache size in bytes: 728
CGAL Polyhedrons in cache: 0
CGAL cache size in bytes: 0
Total rendering time: 0:00:00.000
   Top level object is a 3D object:
   Facets:          6


In [6]:
def read_ascii_stl(filename):
    filename = f"{filename}.stl"

    with open(filename, "r") as f:
        lines = f.readlines()

    vertices = []
    for line in lines:
        parts = line.split()
        if len(parts) > 0 and parts[0] == "vertex":
            vertices.append(list(map(float, parts[1:])))

    faces = np.array(vertices).reshape((-1, 3, 3))
    return faces

In [7]:
facesOne = read_ascii_stl(modelOneFileName)
facesTwo = read_ascii_stl(modelTwoFileName)

print(facesOne)

[[[0. 1. 1.]
  [1. 0. 1.]
  [1. 1. 1.]]

 [[1. 0. 1.]
  [0. 1. 1.]
  [0. 0. 1.]]

 [[0. 0. 0.]
  [1. 1. 0.]
  [1. 0. 0.]]

 [[1. 1. 0.]
  [0. 0. 0.]
  [0. 1. 0.]]

 [[0. 0. 0.]
  [1. 0. 1.]
  [0. 0. 1.]]

 [[1. 0. 1.]
  [0. 0. 0.]
  [1. 0. 0.]]

 [[1. 0. 1.]
  [1. 1. 0.]
  [1. 1. 1.]]

 [[1. 1. 0.]
  [1. 0. 1.]
  [1. 0. 0.]]

 [[1. 1. 0.]
  [0. 1. 1.]
  [1. 1. 1.]]

 [[0. 1. 1.]
  [1. 1. 0.]
  [0. 1. 0.]]

 [[0. 0. 0.]
  [0. 1. 1.]
  [0. 1. 0.]]

 [[0. 1. 1.]
  [0. 0. 0.]
  [0. 0. 1.]]]


In [8]:
def unique_vertices(faces):
    return np.unique(faces.reshape(-1, 3), axis=0)

In [9]:
verticesOne = unique_vertices(facesOne)
verticesTwo = unique_vertices(facesTwo)

In [10]:
def translate_faces(faces, vector):
    return faces + vector

In [11]:
def centroid(faces):
    return np.mean(faces.reshape(-1, 3), axis=0)

In [12]:
def object_borders(faces):
    return np.array(
        [
            [
                np.min(faces[:, :, 0]),
                np.max(faces[:, :, 0]),
            ],
            [
                np.min(faces[:, :, 1]),
                np.max(faces[:, :, 1]),
            ],
            [
                np.min(faces[:, :, 2]),
                np.max(faces[:, :, 2]),
            ],
        ]
    )

In [13]:
print(object_borders(facesOne))

[[0. 1.]
 [0. 1.]
 [0. 1.]]


In [14]:
def visualize_stl_objects(*args, title="", size=None):
    fig = go.Figure()

    for faces in args:
        for face in faces:
            x, y, z = face.T
            fig.add_trace(
                go.Scatter3d(
                    x=np.concatenate([x, [x[0]]]),
                    y=np.concatenate([y, [y[0]]]),
                    z=np.concatenate([z, [z[0]]]),
                    mode="lines+markers",
                )
            )

    if size is not None:
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[0, size[0]]),
                yaxis=dict(range=[0, size[1]]),
                zaxis=dict(range=[0, size[2]]),
                aspectmode="cube",
            )
        )

    fig.update_layout(
        scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"), title=title
    )

    fig.show()

In [15]:
def center_object_xy(faces, center):
    object_center = centroid(faces)
    center = np.array([center[0], center[1], 0])
    return translate_faces(faces, center - object_center)

In [16]:
size_x = 8
size_y = 8
size_z = 4

size = np.array([size_x, size_y, size_z])
center = size / 2

In [17]:
facesOne = center_object_xy(facesOne, center)


visualize_stl_objects(
    facesOne,
    facesTwo,
    title="",
    size=[size_x, size_y, size_z],
)

In [18]:
def aabb(facesOne, facesTwo):
    bordersOne = object_borders(facesOne)
    bordersTwo = object_borders(facesTwo)

    for i in range(3):
        if bordersOne[i][0] > bordersTwo[i][1] or bordersOne[i][1] < bordersTwo[i][0]:
            return False
    return True

In [26]:
newFaces = translate_faces(facesTwo, [1, 0, 0])

print(aabb(newFaces, facesTwo))

visualize_stl_objects(newFaces, facesTwo, title="", size=None)

True


In [20]:
def distance_points(pointOne, pointTwo):
    return np.linalg.norm(pointOne - pointTwo)

In [21]:
print(distance_points(centroid(newFaces), centroid(facesTwo)))

2.0615528128088303


In [27]:
from TriangleInteresects import tri_tri_intersect_with_isectline


def faces_intersection(faces1, faces2):
    for face1 in faces1:
        for face2 in faces2:
            if tri_tri_intersect_with_isectline(
                face1[0], face1[1], face1[2], face2[0], face2[1], face2[2]
            ):
                return True
    return False

In [28]:
print(faces_intersection(newFaces, facesTwo))

True


In [30]:
print(size)
print(object_borders(facesTwo))

[8 8 4]
[[0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
def object_in_plane(faces, size):
  borders = object_borders(faces)

  if 